# HIPPO for 2A Protease (6/?)

## Imports

In [1]:
%load_ext autoreload

In [4]:
%autoreload 2
from pathlib import Path
import molparse as mp
import hippo2 as hippo
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pprint import pprint
import json
import mout
import pickle
import json

## Load the project

In [3]:
# animal = hippo.HIPPO.from_pickle('pickles/2A_hippo_fingerprinted_trim.pickle')
animal = hippo.HIPPO.from_pickle('pickles/2A_hippo_random_latest.pickle')

path = pickles/2A_hippo_random_latest.pickle 


## Load the BBSs

In [5]:
bbs_dir = Path('bbs_5k')

In [7]:
jsons = bbs_dir.glob('BBS_???.json')
bb_sets = []
for i,p in enumerate(jsons):
    # if i > 10:
        # break
    if i%50 == 0:
        mout.out(f'{i=}')
    p = json.load(open(p, 'rt'))
    bb_sets.append(p)

i=0                                                     
i=50                                                    
i=100                                                   
i=150                                                   
i=200                                                   
i=250                                                   
i=300                                                   
i=350                                                   
i=400                                                   
i=450                                                   
i=500                                                   
i=550                                                   
i=600                                                   
i=650                                                   
i=700                                                   
i=750                                                   
i=800                                                   
i=850                          

In [9]:
bb_sets[0]['products'][0]['base']

{'name': 'matteo-ferla-rocs-mixed-Z420031520-1-of-1-step-base',
 'smiles': 'Cc1ccccc1CNC(=O)c1scnc1C',
 'orig_smiles': 'Cc1ccccc1CNC(=O)c1scnc1C',
 'crystal_name': None,
 'num_poses': 0,
 'num_reactions': 1,
 'reactions': [{'type': 'Amidation',
   'product_name': 'matteo-ferla-rocs-mixed-Z420031520-1-of-1-step-base',
   'product_smiles': 'Cc1ccccc1CNC(=O)c1scnc1C',
   'num_reactants': 2,
   'reactant_names': ['Z57474978', 'EN300-11614'],
   'reactant_smiles': ['Cc1ncsc1C(=O)O', 'Cc1ccccc1CN']}],
 'inspirations': ['x0451_0A',
  'x0554_0A',
  'x0556_0A',
  'x0566_0A',
  'x0310_0A',
  'x0416_0A']}

## Define the scorer

In [10]:
def num_bases(bbs):
    # print(bbs.products)
    return len(set([p['base']['name'] for p in bbs['products'] if p['base'] is not None]))

def num_products(bbs):
    return len(bbs['products'])

In [11]:
catalytic_poses = animal.get_poses('Catalytic', search=True, all_poses=animal.hits.poses)
print(catalytic_poses)
CATALYTIC_FEATURES = catalytic_poses.get_present_features()
N_CATALYTIC_FEATURES = len(CATALYTIC_FEATURES)

PoseSet(#poses=8, [0A, 0A, 0A, 0A, 0A, 0A, 0A, 0A])


In [ ]:
def catalytic_features(bbs):
    features = bbs.products.get_present_features()
    return len([f for f in CATALYTIC_FEATURES if f in features])

def num_features(bbs):
    features = bbs.products.get_present_features()
    return len(features)

In [ ]:
print('start')
scorer = hippo.scoring.Scorer(
    animal, 
    bb_sets, 
    [])
    # ['num_products'])

print('num_products')
scorer.add_attribute(hippo.scoring.CustomAttribute('num_products', bb_sets, num_products))
print('num_bases')
scorer.add_attribute(hippo.scoring.CustomAttribute('num_bases', bb_sets, num_bases))
print('catalytic_features')
scorer.add_attribute(hippo.scoring.CustomAttribute('catalytic_features', bb_sets, catalytic_features))
print('num_features')
scorer.add_attribute(hippo.scoring.CustomAttribute('num_features', bb_sets, num_features))

scorer.weights = [1.0, 1.0, 1.0, 1.0]

In [ ]:
top = scorer.top(5)

In [ ]:
for bbs in top:
    scorer.score(bbs, verbosity=1)

## Plotting

In [ ]:
plot_data = []
for bbs in bb_sets:
    d = dict(
        name=bbs.name,
        score=scorer(bbs),
    )

    for attribute in scorer.attributes:
        d[attribute.key] = attribute.get_value(bbs)

    plot_data.append(d)

In [ ]:
px.scatter(plot_data, x='num_products', y='num_features', color='score')

## Write CSV